In [1]:
import pandas as pd
import jieba
import re

In [2]:
df = pd.read_csv('../data/allcontent_AppStore_Soul.csv')
df.head()

,title,content,rating,version,votecount,votesum,time,page
0,可不可以恢复我的账号啊！！！！！！！！,为什么我的账号直接被注销了？不是有七天缓冲期的吗！！！！我还没和朋友说再见啊！！怎么办啊！！！！！,1,4.20.0,0,0,2022-03-05T10:54:42-07:00,1
1,垃圾软件，几年了还这么不稳定,到处是bug，不管是视频，还是语音，没有不出bug的地方，不管你是外放还是用耳机，全部容易有...,1,4.20.0,0,0,2022-03-05T10:14:28-07:00,1
2,环境恶劣,官方能不能设置一个考题，只有完成考题才可以注册聊天，挺好的一个交友软件环境太差了，很影响心情...,1,4.20.0,0,0,2022-03-05T08:59:41-07:00,1
3,太🐮了,感谢🙏soul 让我认识了知心大姐姐,5,4.20.0,0,0,2022-03-05T07:44:16-07:00,1
4,不推荐,平平无奇的找不到，匹配到了聊不下去，然后每天后台挂着还费很多电，垃圾。。。,1,4.20.0,0,0,2022-03-05T06:10:10-07:00,1


In [3]:
df_sub = df[['content','rating']]

In [4]:
df_sub

,content,rating
0,为什么我的账号直接被注销了？不是有七天缓冲期的吗！！！！我还没和朋友说再见啊！！怎么办啊！！！！！,1
1,到处是bug，不管是视频，还是语音，没有不出bug的地方，不管你是外放还是用耳机，全部容易有...,1
2,官方能不能设置一个考题，只有完成考题才可以注册聊天，挺好的一个交友软件环境太差了，很影响心情...,1
3,感谢🙏soul 让我认识了知心大姐姐,5
4,平平无奇的找不到，匹配到了聊不下去，然后每天后台挂着还费很多电，垃圾。。。,1
...,...,...
495,同步后账户一致但无法更新,1
496,上面很多涉黄的话图片没看你们平台处理，我就发了一句“只要你缺钱咱俩就有缘”居然被封号！简直了,1
497,以后看着吧,5
498,倾听你的声音，倾诉你的内心。净化你的心灵，静静等待你的到来。,5


In [63]:
stoptext = open('../cn_stopwords.txt', encoding='utf-8').read()
stopwords = stoptext.split('\n')
def filter_str(desstr, restr=''):
    # 过滤除中英文及数字以外的其他字符
    res = re.compile("[^\\u4e00-\\u9fa5^a-z^A-Z^0-9]")
    return res.sub(restr, desstr)

def clean_text(text):
    words = jieba.lcut(text)
    words = [w for w in words if w not in stopwords]
    return ' '.join(words)


In [38]:
df_sub['content_words'] = df_sub['content'].apply(filter_str).apply(clean_text)

In [39]:
df_sub.head()

,content,rating,content_words
0,为什么我的账号直接被注销了？不是有七天缓冲期的吗！！！！我还没和朋友说再见啊！！怎么办啊！！！！！,1,账号 直接 注销 七天 缓冲期 没 朋友 说 再见
1,到处是bug，不管是视频，还是语音，没有不出bug的地方，不管你是外放还是用耳机，全部容易有...,1,到处 bug 视频 语音 没有 不出 bug 地方 外放 耳机 容易 回声 网络 明明 没 ...
2,官方能不能设置一个考题，只有完成考题才可以注册聊天，挺好的一个交友软件环境太差了，很影响心情...,1,官方 不能 设置 考题 完成 考题 注册 聊天 挺 交友 软件 环境 太差 影响 心情 半夜...
3,感谢🙏soul 让我认识了知心大姐姐,5,感谢 soul 认识 知心 姐姐
4,平平无奇的找不到，匹配到了聊不下去，然后每天后台挂着还费很多电，垃圾。。。,1,平平 无奇 找 不到 匹配 聊 下去 每天 后台 挂 还费 很多 电


In [40]:
from bertopic import BERTopic
import spacy

In [41]:
zh_model = spacy.load("zh_core_web_sm")
topic_model = BERTopic(language="chinese (simplified)", 
                       embedding_model=zh_model,
                       calculate_probabilities=True, 
                       verbose=True)

In [42]:
df_sub = df_sub[df_sub['content_words']!='']

In [43]:
docs = df_sub['content_words'].tolist()
docs

['账号 直接 注销 七天 缓冲期 没 朋友 说 再见',
 '到处 bug 视频 语音 没有 不出 bug 地方 外放 耳机 容易 回声 网络 明明 没 问题 非 显示 网络 不佳',
 '官方 不能 设置 考题 完成 考题 注册 聊天 挺 交友 软件 环境 太差 影响 心情 半夜 气 睡不着 觉',
 '感谢 soul 认识 知心 姐姐',
 '平平 无奇 找 不到 匹配 聊 下去 每天 后台 挂 还费 很多 电',
 '老 用户 太 喷子 软件 应该 怼 怼 没事 评论 诋毁 抑郁症 医生 社交 软件 用户 真的 滴 玩 气死 劝 手',
 '女性 玩家 app 保护 女性 玩家 狼 杀 功能 房间 没事 当众 骂 女 女生 号 封 封 男 号 不分 黑白 软件',
 '里面 好多 人妖',
 '玩 这么久 没什么 变化 充值 商品 不少 莫名 禁言 封号 无语 社交 软件 用户 带来 体验 应该 有趣 充满 期待 一昧 圈钱 懂得 留住 用户 用户 反馈 得不到 及时 合理 解决 实在 失望',
 '不可 传 原图 上传 图片 加 层 滤镜',
 'APP 真的 不能 没有 好人 不说 上面 卖 没事 想 知道 官方 管理 坏人 没 整治 好人 报废',
 '没有 编辑 发送 瞬间 功能',
 '开发人员 脑子 想 东西 开屏 广告 没点 转 对应 app 直接 打开 恰烂 钱',
 '太 人性化 登录 不了 老 提示 异常 勒设 软件',
 '软件 收费 勇气 收费 要饭',
 '软件 三天 一小 崩溃 五天 一大 崩溃 不错',
 '下载 不得好死',
 '太 社交 软件 语音 经常 掉线 知道 最后 落地 微信',
 '标签 爵士乐 另一人 标签 民谣 麻烦 不要 推荐 真的 当头一棒 鄙视 真的 无法 喜欢 民谣 匹配 令 更加 反感 感性 女生 想法 采纳',
 '动不动 封',
 '里面 充值 页面 网络 暴力 现在 卸载 天天 发短信 下载',
 '真的 没有 不当 言论 直接 强制 掉线 强制 封号',
 '结交 更 新 朋友',
 'UI 丑',
 '下载 钱',
 '红色 界面 喜欢',
 '明天 坚持 开 派对 公益 解决问题 最近 老 限流 学习 进不了 房间',
 '没有 超级 可爱 爱 表情 有趣 人太少',
 '希望 脱单',


In [44]:
topics, probs = topic_model.fit_transform(docs)

100%|██████████| 490/490 [00:02<00:00, 165.43it/s]
2022-03-07 17:31:40,074 - BERTopic - Transformed documents to Embeddings
2022-03-07 17:31:42,221 - BERTopic - Reduced dimensionality with UMAP
2022-03-07 17:31:42,281 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [45]:
topic_model.get_topic_info()


,Topic,Count,Name
0,0,442,0_软件_soul_没有_匹配
1,1,38,1_软件_app_溜达_好用
2,-1,10,-1_开心_超过_有喜_变黑


In [46]:
topic_model.get_topic(0)

[('软件', 0.07154459005878186),
 ('soul', 0.04653674342228405),
 ('没有', 0.04539137312153414),
 ('匹配', 0.04218552739241986),
 ('真的', 0.034104343012021074),
 ('封号', 0.02867911113006485),
 ('用户', 0.02772977289080934),
 ('知道', 0.02753794383478677),
 ('聊天', 0.026567509776568948),
 ('不能', 0.025978036156908205)]

In [47]:
topic_model.visualize_distribution(probs[0])


In [48]:
topic_model.visualize_term_rank() 

In [49]:
df_highscore = df[df['rating']>=4]

In [50]:
df_highscore['content_words'] = df_highscore['content'].apply(filter_str).apply(clean_text)

In [58]:
docs = ' '.join(df_highscore['content_words'].tolist())

In [60]:
import jieba.analyse
topK = 20
tags = jieba.analyse.extract_tags(docs, topK=topK,withWeight=True)
tags

[('soul', 0.29311163754921193),
 ('软件', 0.15448743067754817),
 ('不错', 0.06502522400385288),
 ('开心', 0.06409037644975482),
 ('感谢', 0.052484405330437835),
 ('认识', 0.04973264061339754),
 ('真的', 0.04933224434649737),
 ('社交', 0.04916313241528897),
 ('朋友圈', 0.04383321643047285),
 ('希望', 0.041022806021558667),
 ('朋友', 0.040960479739684766),
 ('匹配', 0.04090079961378284),
 ('喜欢', 0.039948079881050785),
 ('很多', 0.0382876918017951),
 ('遇到', 0.037822697020236426),
 ('游戏', 0.03777024828504378),
 ('下载', 0.03755730617774081),
 ('非常', 0.034348742601330993),
 ('一下', 0.03425062096406305),
 ('页面', 0.03417703301429072)]

In [66]:
df_lowscore = df[df['rating']<=2]
df_lowscore['content_words'] = df_lowscore['content'].apply(filter_str).apply(clean_text)
docs = ' '.join(df_lowscore['content_words'].tolist())
topK = 20
tags = jieba.analyse.extract_tags(docs, topK=topK,withWeight=False)
tags

['匹配',
 '封号',
 '聊天',
 '举报',
 '真的',
 '卸载',
 '用户',
 'app',
 '账号',
 '没有',
 '下载',
 '恶心',
 '语音',
 '申诉',
 '社交',
 '违规',
 '禁言',
 '注册',
 '充钱',
 '直接']

In [68]:
def keyword_in_sentence(sentence,keywords = tags):
    for i in keywords:
        if i in sentence:
            return 1
        
    return 0

df_keyword = df[df['content'].apply(keyword_in_sentence)==1]

In [69]:
df_keyword

,title,content,rating,version,votecount,votesum,time,page
0,可不可以恢复我的账号啊！！！！！！！！,为什么我的账号直接被注销了？不是有七天缓冲期的吗！！！！我还没和朋友说再见啊！！怎么办啊！！！！！,1,4.20.0,0,0,2022-03-05T10:54:42-07:00,1
1,垃圾软件，几年了还这么不稳定,到处是bug，不管是视频，还是语音，没有不出bug的地方，不管你是外放还是用耳机，全部容易有...,1,4.20.0,0,0,2022-03-05T10:14:28-07:00,1
2,环境恶劣,官方能不能设置一个考题，只有完成考题才可以注册聊天，挺好的一个交友软件环境太差了，很影响心情...,1,4.20.0,0,0,2022-03-05T08:59:41-07:00,1
4,不推荐,平平无奇的找不到，匹配到了聊不下去，然后每天后台挂着还费很多电，垃圾。。。,1,4.20.0,0,0,2022-03-05T06:10:10-07:00,1
5,不值得玩,我也是老用户了，看了太多喷子了，这个软件应该叫怼怼。这些人没事就评论一些诋毁你的话，有抑郁症...,1,4.20.0,0,0,2022-03-05T04:51:13-07:00,1
...,...,...,...,...,...,...,...,...
487,搜索昵称搜不到人！！！！,我真的要爆哭了，由于之前跟soulmate产生矛盾大吵一架之后我拉黑了他，后续我很长一段时间...,2,4.18.0,0,0,2022-01-31T02:00:22-07:00,10
488,主要为了上来,真的服了，我当个管理啥也没干被封了，真的服了，说我低俗色情，什么啊我根本没有，然后封了我五天...,5,4.18.0,0,0,2022-01-31T01:34:50-07:00,10
489,恶臭,里面全是恶臭蝻，随便乱封号，纵容色情骚扰,1,4.18.0,0,0,2022-01-30T21:09:29-07:00,10
493,垃圾,手机号码都没有注册过，刚刚去注册显示已经给别人注册了，别人注册我的手机号码连验证码都不用吗？真垃圾,1,4.18.0,0,0,2022-01-30T09:10:53-07:00,10


In [70]:
df_keyword.to_csv('../data/lowscore_keyword_content.csv',encoding='utf-8-sig',index=None)